# CTGAN Model

In this guide we present the `CTGAN` model: A GAN-based Deep Learning data synthesizer that can generate synthetic tabular data with high  fidelity. Based on our [CTGAN Library](https://github.com/sdv-dev/CTGAN).

<div class="alert alert-info">

**NOTE**

During this guide we will walk you through the specific functionalities of the `CTGAN` model.
For a more generic view of the SDV Tabular Models and their common functionalities, please visit
the [Tabular Models](01_Tabular_Models.ipynb) guide.

</div>

## Modeling Tabular data using Conditional GAN

**CTGAN** is a Deep Learning based data syntheziser that uses Generative Adversarial Networks
to generate tabular data and which was presented at the NeurIPS 2020 conference by the
paper titled [Modeling Tabular data using Conditional GAN](https://arxiv.org/abs/1907.00503).
For more details about it, please read the linked paper and visit the [CTGAN library](
https://github.com/sdv-dev/CTGAN).

Let's now discover how to learn a dataset and later on generate synthetic data with the same
format and statistical properties by using the `CTGAN` class from SDV.

<div class="alert alert-warning">

WARNING

In order to follow this guide you need to have `ctgan` installed on your system.
If you have not done it yet, please install `ctgan` now by executing the command
`pip install sdv[ctgan]` in a terminal.

</div>

## Introducing CTGAN

We will start by loading one of our demo datasets, the `student_placements`, which we used during
[Tabular Models](01_Tabular_Models.ipynb) guide.

In [1]:
# Setup logging and warnings - change ERROR to INFO for increased verbosity
import logging;
logging.basicConfig(level=logging.ERROR)

logging.getLogger().setLevel(level=logging.WARNING)
logging.getLogger('sdv').setLevel(level=logging.ERROR)

import warnings
warnings.simplefilter("ignore")

In [2]:
from sdv.demo import load_tabular_demo

data = load_tabular_demo('student_placements')
data.head().T

,0,1,2,3,4
student_id,155368,155369,155370,155371,155372
gender,M,M,M,M,M
second_perc,67,79.33,65,56,85.8
high_perc,91,78.33,68,52,73.6
high_spec,Commerce,Science,Arts,Science,Commerce
degree_perc,58,77.48,64,52,73.3
degree_type,Sci&Tech,Sci&Tech,Comm&Mgmt,Sci&Tech,Comm&Mgmt
work_experience,False,True,False,False,False
experience_years,0,1,0,0,0
employability_perc,55,86.5,75,66,96.8


As we learned in the [Tabular Models](01_Tabular_Models.ipynb) guide, the first
step that we need to do in order to use tabular model like `CTGAN` is to import
its class and create an instance of it passing the details about our data.

In this case, we would only need to indicate that the primary key is the `student_id`
field and call its `fit` method.

In [3]:
from sdv.tabular import CTGAN

model = CTGAN(
    primary_key='student_id',
)
model.fit(data)

After this is done, we can simply call its `sample` method to obtain
syntehtically generated data from it:

In [4]:
new_data = model.sample(len(data))

In [5]:
new_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,F,55.673440,35.385329,Science,77.910653,Sci&Tech,False,0,67.267717,Mkt&HR,57.796472,26961.957338,True,NaT,2020-03-30 11:29:56.959432960,NaN
1,1,M,57.898309,53.293088,Commerce,67.198933,Comm&Mgmt,False,1,73.089297,Mkt&HR,66.888349,NaN,True,2020-02-16 00:03:24.176280576,NaT,6.0
2,2,M,59.027578,37.784593,Science,66.922251,Sci&Tech,False,1,74.090553,Mkt&HR,66.220386,17684.222919,True,2019-12-19 21:22:16.301072128,2020-12-08 09:33:28.105755136,6.0
3,3,M,50.055418,38.609832,Arts,78.842213,Comm&Mgmt,True,0,95.951477,Mkt&HR,73.716787,32224.660930,False,2020-01-23 14:08:04.391622912,2020-03-10 22:05:46.568782592,NaN
4,4,F,49.980109,60.661264,Commerce,76.132861,Comm&Mgmt,False,0,55.252820,Mkt&HR,60.326783,28482.033544,True,2019-12-12 04:15:40.063726592,2020-10-19 10:05:23.907214336,NaN


## CTGAN Hyperparameters

A part from the common Tabular Model arguments, `CTGAN` has a number of additional
hyperparameters that control its learning behavior and can impact on the
performance of the model, both in terms of quality of the generated data
and computationa time.

### epochs and batch size

The first hyperparameters that we see are the `epochs` and `batch_size` arguments,
which control the number of iterations that the model will perform to optimize
its parameters, as well as the number of samples used in each step.

Its default values are `300` and `500` respectively, and `batch_size` needs to
always be a value which is multiple of `10`.

These hyperparameters have a very direct effect in time the training process lasts,
but also on the performance of the data.

For new datasets, you might want to start by setting a low value on both of them
to see how long the training process takes on your data and later on increase the number
to acceptable values in order to improve the performance.

### log_frequency

Whether to use log frequency of categorical levels in conditional sampling.

Defaults to `True`.

This argument affects how the model processes the frequencies of the categorical
values that are used to condition the rest of the values. In some cases,
changing it to `False` could lead to better performance.

### Neural Network dimensions

`CTGAN` has the following hyperparameters that allow you to control the
size of the different layers that compose its neural networks:

- embedding_dim (int): Size of the random sample passed to the Generator. Defaults to 128.
- gen_dim (tuple or list of ints): Size of the output samples for each one of the Residuals.
  A Resiudal Layer will be created for each one of the values provided. Defaults to (256, 256).
- dis_dim (tuple or list of ints): Size of the output samples for each one of the Discriminator
  Layers. A Linear Layer will be created for each one of the values provided. Defaults to (256, 256).

### l2scale

The `l2scale` argument, which defaults to `1e-6`, sets the wheight Decay of the Adam Optimizer
used to optimize the Neural Networks.

### verbose

Whether to print fit progress on stdout. Defaults to `False`.

<div class="alert alert-warning">
    
**WARNING**
    
The value that you set on the `batch_size` argument must always be
a multiple of `10`!

</div>

As an example, we will try to fit the `CTGAN` model slightly increasing the number of epochs,
reducing the `batch_size`, adding one additional layer to the models involved and using a
smaller wright decay.

Before we start, we will evaluate the qualtiy of the previously generated data using the
`sdv.evaluation.evaluate` function

In [6]:
from sdv.evaluation import evaluate

evaluate(new_data, data)

-148.173551500529

Afterwards, we create a new instance of the `CTGAN` model with the
hyperparameter values that we want to use

In [7]:
model = CTGAN(
    primary_key='student_id',
    epochs=500,
    batch_size=100,
    gen_dim=(256, 256, 256),
    dis_dim=(256, 256, 256),
    l2scale=1e-07
)

And fit to our data.

In [8]:
model.fit(data)

Finally, we are ready to generate new data and evaluate the results.

In [9]:
new_data = model.sample(len(data))

In [10]:
new_data

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,F,34.949775,56.543753,Science,56.386929,Others,True,0,57.047205,Mkt&HR,70.880870,52487.953908,False,2020-02-11 14:51:51.540146176,NaT,NaN
1,1,M,60.293816,63.604466,Commerce,57.213664,Comm&Mgmt,False,1,76.443608,Mkt&Fin,69.989944,49340.176244,False,NaT,2020-08-22 07:51:09.761925888,3.0
2,2,M,65.549707,67.099558,Science,67.883755,Comm&Mgmt,False,0,63.329463,Mkt&HR,77.980852,NaN,False,NaT,NaT,6.0
3,3,M,31.861321,51.808897,Commerce,59.313310,Comm&Mgmt,False,4,91.136983,Mkt&Fin,84.226341,NaN,False,NaT,2020-07-21 22:16:00.456667392,12.0
4,4,F,78.827052,75.281691,Commerce,64.924865,Comm&Mgmt,False,1,68.389091,Mkt&Fin,80.486051,51901.492908,False,2020-01-25 13:18:42.237984512,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,210,M,67.010665,43.334043,Science,58.679588,Comm&Mgmt,False,0,89.946393,Mkt&Fin,78.518156,NaN,False,NaT,2020-10-02 00:22:02.203142656,NaN
211,211,M,60.100223,68.346138,Science,72.964973,Sci&Tech,False,0,101.124253,Mkt&Fin,76.647721,32727.642571,False,2020-02-13 20:57:41.302214656,2021-01-03 03:54:04.681563904,12.0
212,212,M,54.662673,56.751646,Commerce,60.198562,Comm&Mgmt,True,0,50.684929,Mkt&HR,66.078017,NaN,False,2020-03-01 10:52:01.466955776,2020-08-19 20:15:37.505603072,12.0
213,213,M,57.984654,51.599140,Science,69.113537,Comm&Mgmt,False,0,61.562135,Mkt&HR,72.756443,39093.193199,False,2020-02-08 20:32:23.699396864,NaT,NaN


In [11]:
from sdv.evaluation import evaluate

evaluate(new_data, data)

-153.27980312716866

As we can see, in this case these modifications changed the obtained results slightly,
but they did neither introduce dramatic changes in the performance.